In this notebook, we walk through how to prepare the "annotation files" required by tagtools to annotate the reads with transcript IDs and names. 

The starting point is a GFF3 file for the species of interest. GFF3 files for different species and from different databases are organized somewhat differently, so one needs to modify the steps below according to the specifics of the GFF# file. 

Here, we generate the annotation files for a Drosphila using a GFF3 file downloaded from Encode. 

## Drosophila, GFF3 from Encode

### Download GFF3
This is the file to download
<ftp://ftp.ensembl.org/pub/release-95/gff3/drosophila_melanogaster/Drosophila_melanogaster.BDGP6.95.gff3.gz>


### Make an annotation table

First we need to make a simple "annotation" table, which is a simple text file which links a transcript ID to a gene name, type, strand and length. This table will be used by tagtools for annotating the reads with actual transcript id and name. The file should look like this

```txt
ENST00000000233.9       ARF5    protein_coding  +       3360
ENST00000000412.7       M6PR    protein_coding  -       9590
ENST00000000442.10      ESRRA   protein_coding  +       11160
```

For the human gff file, we can do that with the following command

```bash
awk -F $'\t' 'BEGIN{OFS="\t"}(/^#/){next;}($3=="transcript"){split($9, x,";"); for (i = 0; ++i <= length(x);){delete x; delete y; delete z; split(x[i],y,"="); z[y[1]]=y[2];}; s=$4; s2=$5; print z["ID"], z["gene_name"], z["gene_type"], $7, s2-s}' gencode.v29.primary_assembly.annotation.gff3
```

For dmel, the desired features are not labeled as transcript, as you can see with 

```bash
cat Drosophila_melanogaster.BDGP6.95.gff3 | grep -v "##" | grep "ID=transcript" | cut -f3 | sort | uniq
```

Therefore the proper command is 

```bash
awk -F $'\t' 'BEGIN{types["mRNA"]=1; types["ncRNA"]=1; types["pre_miRNA"]=1; types["pseudogenic_transcript"]=1; types["rRNA"]=1; types["snRNA"]=1; types["snoRNA"]=1; types["tRNA"]=1; OFS="\t"}(/^#/){next;}($3 in types){delete x; delete y; delete z; split($9, x,";"); for (i = 0; ++i <= length(x);){split(x[i],y,"="); z[y[1]]=y[2];}; s=$5-$4; print substr(z["ID"],12), z["Name"], z["biotype"], $7, s, substr(z["Parent"],6)}' Drosophila_melanogaster.BDGP6.95.gff3 > tableGENES.withStrand.txt
```

### Make the exons dictionnary
In order to convert transcriptome coordinates to genomic coordinates, tagtools require a python dictionnary which contains the location of the splice junctions for each transcript. For now, we are doing this manually by adjusting the definition of the function below to account for the features of the specific gtf file we are interested in. This function will create a python dictionnary which which we will save as a pickle file (this will be changed into json file in a future release) 

In [41]:
import subprocess
import numpy as np
def make_exons_dict_fromGFF(gff_file, nmax=0):
    m={}
    
    cmd = "cat "+gff_file+""" | awk -F $'\t' 'BEGIN{types["mRNA"]=1; types["ncRNA"]=1; types["pre_miRNA"]=1; types["pseudogenic_transcript"]=1; types["rRNA"]=1; types["snRNA"]=1; types["snoRNA"]=1; types["tRNA"]=1; OFS="\t"}(/^#/){next;}($3 in types)"""
    cmd+="""{delete xarr; delete yy; delete zz; x=$9; split(x,xarr,\";\"); for (i=1; i<=length(xarr); i++)"""
    cmd+=""" {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print zz["transcript_id"], $4, $5, $7, $1}'"""
    
    print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, bufsize=1, universal_newlines=True, start_new_session=True)
    nok=0
    for _, line in enumerate(p.stdout):
        if nmax>0 and nok>nmax:
            break
        read_data=line.strip().split("\t") 
        T_id=read_data[0] #transcript I
        
        if T_id in m:
            print("oops, already found")
#             m[T_id]+=[read_data[3],read_data[1],read_data[2]]
        else:
            nok+=1
            readstrand=int(1) if read_data[3]=="+" else int(-1)
            m[T_id]=[readstrand,int(read_data[1]),int(read_data[2]),read_data[4]]
            
    p.terminate()
    del p
    
    m2={}
    cmd = "cat "+gff_file+""" | awk -F $'\t' 'BEGIN{OFS="\t"}((!/^#/) && $3=="exon")"""
    cmd+="""{delete xarr; delete yy; delete zz; x=$9; split(x,xarr,\";\"); for (i=1; i<=length(xarr); i++)"""
    cmd+=""" {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print substr(zz["Parent"],12), $4, $5}'"""
    print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, bufsize=1, universal_newlines=True, start_new_session=True)
    nok=0
    for _, line in enumerate(p.stdout):
        if nmax>0 and nok>nmax:
            break
        read_data=line.strip().split("\t") 
        T_id=read_data[0] #transcript I
        
        vals=m2.pop(T_id,[])
        if len(vals)>0:
            
            m2[T_id]=np.vstack([vals,np.array([int(read_data[1]),int(read_data[2])])])
        else:
            nok+=1
            m2[T_id]=np.array([[int(read_data[1]),int(read_data[2])]])
    
    p.terminate()
    del p
    
    for k, v in m.items():
        vals=m2.pop(k,[])
        if len(vals)>0:
            if v[0]==1:
                a=np.argsort(vals[:,0])
                m2[k]=[v[3],np.insert(np.cumsum(vals[a,1]-vals[a,0]+1),0,0),vals[a,0]]
            else:
                a=np.argsort(-vals[:,1])
                m2[k]=[v[3],np.insert(np.cumsum(vals[a,1]-vals[a,0]+1),0,0),-vals[a,1]]
        else:
            m2[k]=[]
    
    return m2

In [33]:
dmel_exons=make_exons_dict_fromGFF("Drosophila_melanogaster.BDGP6.95.gff3")

cat ../tmp/Drosophila_melanogaster.BDGP6.95.gff3 | awk -F $'	' 'BEGIN{types["mRNA"]=1; types["ncRNA"]=1; types["pre_miRNA"]=1; types["pseudogenic_transcript"]=1; types["rRNA"]=1; types["snRNA"]=1; types["snoRNA"]=1; types["tRNA"]=1; OFS="	"}(/^#/){next;}($3 in types){delete xarr; delete yy; delete zz; x=$9; split(x,xarr,";"); for (i=1; i<=length(xarr); i++) {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print zz["transcript_id"], $4, $5, $7, $1}'
cat ../tmp/Drosophila_melanogaster.BDGP6.95.gff3 | awk -F $'	' 'BEGIN{OFS="	"}((!/^#/) && $3=="exon"){delete xarr; delete yy; delete zz; x=$9; split(x,xarr,";"); for (i=1; i<=length(xarr); i++) {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print substr(zz["Parent"],12), $4, $5}'


Verify that the exons dictionnary looks fine

In [34]:
{k:v for i, (k,v) in enumerate(dmel_exons.items()) if i<3}

{'FBtr0076316': ['3L',
  array([   0,  359,  541, 2112]),
  array([-10089400, -10088979, -10088317])],
 'FBtr0112861': ['2R',
  array([   0,   58,  203,  550,  784,  862,  962, 1312]),
  array([18819467, 18819621, 18820196, 18820605, 18821041, 18821184, 18821490])],
 'FBtr0307368': ['3L',
  array([   0,  206,  401,  628,  916, 1028, 1183, 1656, 2042, 2163, 2344,
         2494, 2616, 2709, 2856, 3015, 3160, 3255, 3415, 3513, 3604, 3718,
         3868, 4029, 4163, 4273, 4443, 4625, 4750, 6246]),
  array([-16543370, -16539340, -16530656, -16529914, -16522910, -16521032,
         -16520541, -16518596, -16518146, -16517660, -16516801, -16515978,
         -16514449, -16513480, -16513123, -16511175, -16510966, -16510810,
         -16510434, -16510108, -16509942, -16508614, -16508065, -16507467,
         -16506486, -16505230, -16504941, -16504446, -16503529])]}

Now we can save this dictionnary

In [35]:
import pickle
with open("annotation.gff3.pickle", 'wb') as handle:
    pickle.dump(dmel_exons, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Make an annotation table and exons dictionnary for the gene bodies

We adjust the annotation table generation to get the gene bodies as opposed to transcripts. The genebodies are not just entered as gene, as seen by:

```bash
cat Drosophila_melanogaster.BDGP6.95.gff3 | awk '(substr($9,1,7)=="ID=gene")' | cut -f3,3 | sort | uniq
```

so we need

```bash
awk -F $'\t' 'BEGIN{x["ID"]=1; y[1]=0; z["ID"]=1; types["gene"]=1; types["ncRNA_gene"]=1; types["pseudogene"]=1; OFS="\t"}(/^#/){next;}($3 in types){delete x; delete y; delete z; split($9, x,";"); for (i = 0; ++i <= length(x);){split(x[i],y,"="); z[y[1]]=y[2];}; s=$5-$4; print z["gene_id"], z["Name"], z["biotype"], $7, s}' Drosophila_melanogaster.BDGP6.95.gff3 > tableGENES.withStrand_GENEBODIES.txt
```

And now the exons dictionnary

In [38]:
def make_genebodies_dict_fromGFF(gff_file, nmax=0):
    m={}
    
    cmd = "cat "+gff_file+""" | awk -F $'\t' 'BEGIN{types["gene"]=1; types["ncRNA_gene"]=1; types["pseudogene"]=1; OFS="\t"}(/^#/){next;}($3 in types)"""
    cmd+="""{delete xarr; delete yy; delete zz; x=$9; split(x,xarr,\";\"); for (i=1; i<=length(xarr); i++)"""
    cmd+=""" {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print zz["gene_id"], $4, $5, $7, $1}'"""
    
    print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, bufsize=1, universal_newlines=True, start_new_session=True)
    nok=0
    for _, line in enumerate(p.stdout):
        if nmax>0 and nok>nmax:
            break
        read_data=line.strip().split("\t") 
        T_id=read_data[0] #transcript I
        
        if T_id in m:
            print("oops, already found")
#             m[T_id]+=[read_data[3],read_data[1],read_data[2]]
        else:
            nok+=1
            readstrand=int(1) if read_data[3]=="+" else int(-1)
            m[T_id]=[readstrand,int(read_data[1]),int(read_data[2]),read_data[4]]
            
    p.terminate()
    del p
    
    m2={}
    cmd = "cat "+gff_file+""" | awk -F $'\t' 'BEGIN{types["gene"]=1; types["ncRNA_gene"]=1; types["pseudogene"]=1; OFS="\t"}(/^#/){next;}($3 in types)"""
    cmd+="""{delete xarr; delete yy; delete zz; x=$9; split(x,xarr,\";\"); for (i=1; i<=length(xarr); i++)"""
    cmd+=""" {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print zz["gene_id"], $4, $5}'"""
    print(cmd)
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, bufsize=1, universal_newlines=True, start_new_session=True)
    nok=0
    for _, line in enumerate(p.stdout):
        if nmax>0 and nok>nmax:
            break
        read_data=line.strip().split("\t") 
        T_id=read_data[0] #transcript I
        
        vals=m2.pop(T_id,[])
        if len(vals)>0:
            
            m2[T_id]=np.vstack([vals,np.array([int(read_data[1]),int(read_data[2])])])
        else:
            nok+=1
            m2[T_id]=np.array([[int(read_data[1]),int(read_data[2])]])
    
    p.terminate()
    del p
    
    for k, v in m.items():
        vals=m2.pop(k,[])
        if len(vals)>0:
            if v[0]==1:
                a=np.argsort(vals[:,0])
                m2[k]=[v[3],np.insert(np.cumsum(vals[a,1]-vals[a,0]+1),0,0),vals[a,0]]
            else:
                a=np.argsort(-vals[:,1])
                m2[k]=[v[3],np.insert(np.cumsum(vals[a,1]-vals[a,0]+1),0,0),-vals[a,1]]
        else:
            m2[k]=[]
    
    return m2

In [39]:
dmel_genebodies=make_genebodies_dict_fromGFF("Drosophila_melanogaster.BDGP6.95.gff3")
{k:v for i, (k,v) in enumerate(dmel_genebodies.items()) if i<3}

cat ../tmp/Drosophila_melanogaster.BDGP6.95.gff3 | awk -F $'	' 'BEGIN{types["gene"]=1; types["ncRNA_gene"]=1; types["pseudogene"]=1; OFS="	"}(/^#/){next;}($3 in types){delete xarr; delete yy; delete zz; x=$9; split(x,xarr,";"); for (i=1; i<=length(xarr); i++) {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print zz["gene_id"], $4, $5, $7, $1}'
cat ../tmp/Drosophila_melanogaster.BDGP6.95.gff3 | awk -F $'	' 'BEGIN{types["gene"]=1; types["ncRNA_gene"]=1; types["pseudogene"]=1; OFS="	"}(/^#/){next;}($3 in types){delete xarr; delete yy; delete zz; x=$9; split(x,xarr,";"); for (i=1; i<=length(xarr); i++) {split(xarr[i],yy,"="); zz[yy[1]]=yy[2];}; print zz["gene_id"], $4, $5}'


{'FBgn0003495': ['3R', array([   0, 5081]), array([-27070070])],
 'FBgn0053443': ['2R', array([  0, 135]), array([-19733060])],
 'FBgn0267115': ['3R', array([  0, 698]), array([31473041])]}

Everything looks fine so we can save it

In [40]:
with open("annotation_GENEBODIES.gff3.pickle", 'wb') as handle:
    pickle.dump(dmel_genebodies, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Drosophila, GFF3 from NCBI

### Download GFF3
The GFF file is downloaded from here : <https://www.ncbi.nlm.nih.gov/genome?term=vih&cmd=DetailsSearch>

In progress...